In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('../input/air-passengers/AirPassengers.csv')
dataSize = len(df)
df.head()

In [ ]:
linspace = [0]
while linspace[-1] < 143-12:
    linspace.append(linspace[-1]+12)
linspace.append(143)

plt.plot(df['Month'], df['#Passengers'])
plt.xticks(linspace, rotation='vertical')
plt.xlabel('Time in Months')
plt.ylabel('Number of passengers')

In [ ]:
dataMat = df['#Passengers'].values.reshape(dataSize,1)
dataMat.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaledDataMat = scaler.fit_transform(dataMat)

In [ ]:
train_split = .7
train_size = int(train_split * dataSize)
trainData = scaledDataMat[:train_size]
valData = scaledDataMat[train_size:]
print('Train data size: ', trainData.shape)
print('Val data size: ', valData.shape)

In [ ]:
def createDataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step):
		dataX.append(dataset[i:(i+time_step)])
		dataY.append(dataset[i + time_step])
	return np.array(dataX), np.array(dataY)

In [ ]:
n_past = 4
X_train,y_train = createDataset(trainData, time_step=n_past)
X_val,y_val = createDataset(valData, time_step=n_past)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(n_past,1)))
model.add(LSTM(64, return_sequences=False, input_shape=(n_past,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
history = model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=50,batch_size=1,verbose=1, callbacks=[callback])

In [ ]:
y_train_hat = scaler.inverse_transform(model.predict(X_train))
y_val_hat = scaler.inverse_transform(model.predict(X_val))

In [ ]:
train_shift = n_past
test_shift = train_shift + len(y_train_hat) + n_past

plt.plot(df['Month'], df['#Passengers'], color='b', label="Original Data")
plt.plot(df['Month'][train_shift: test_shift-n_past], y_train_hat, color='r', label="Train Forecast")
plt.plot(df['Month'][test_shift:], y_val_hat, color='y', label="Val Forecast")
plt.legend(loc='best', shadow=True)

plt.xticks(linspace, rotation='vertical')
plt.xlabel('Time in Months')
plt.ylabel('Number of passengers')